In [40]:
import pandas as pd
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET

# WP: Wahlperiode
# Quelle der Daten: 

In [53]:
DATA_PATH = Path('data')
MDB_XML_PATH = DATA_PATH / 'MDB_STAMMDATEN.XML'
DF_MDB_PATH = DATA_PATH / 'df_mdb.csv' #one row per individuum, one col per wp containing 0 / 1
DF_MDB_WP_PATH = DATA_PATH / 'df_mdb_wp.csv' # one row per individuum per wp, containting e.g. 12. so one abgeordneter who has been in parliament for several wp will receive one row per wp

# parse xml data into dataFrame / .csv fomat

In [42]:
def get_mdb_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            mdb_dict[int(wahlperiode.find('WP').text)] = 1

        list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [47]:
def get_mdb_wp_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            # one seperate entry per wp
            mdb_dict['WP']=int(wahlperiode.find('WP').text)
            list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [48]:
df_mdb = get_mdb_data(MDB_XML_PATH)
print(df_mdb.shape)
df_mdb.head()

(4089, 34)


,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,None,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000004,Ackermann,Else,06.11.1933,Berlin,None,14.09.2019,weiblich,ledig,evangelisch,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000005,Adam,Ulrich,09.06.1950,"Teterow, Kr. Teterow, Bezirk Neubrandenburg",None,None,männlich,"verheiratet, 2 Kinder",evangelisch,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [50]:
df_mdb_wp = get_mdb_wp_data()
print(df_mdb_wp.shape)
df_mdb_wp.head()

(11627, 16)


,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,ANZ_WAHLPERIODEN,WP
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
1,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
2,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
3,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
4,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11


# clean data

In [51]:
# ok, quite expected. some cleaning necessarry: 'BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE'
for grünen_alias in ['BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE']:
    df_mdb.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)
    df_mdb_wp.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)

# save as csv

In [54]:
df_mdb.to_csv(DF_MDB_PATH, index=None)
df_mdb_wp.to_csv(DF_MDB_WP_PATH, index=None)

# read in data. exploratory data analysis

In [55]:
df_mdb = pd.read_csv(DF_MDB_PATH, index_col=None)

In [56]:
df_mdb.columns

Index(['ID', 'NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'GEBURTSORT',
       'GEBURTSLAND', 'STERBEDATUM', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION',
       'BERUF', 'PARTEI_KURZ', 'VITA_KURZ', 'VEROEFFENTLICHUNGSPFLICHTIGES',
       'ANZ_WAHLPERIODEN', '5', '6', '7', '8', '9', '10', '11', '3', '4', '2',
       '12', '13', '14', '15', '16', '1', '17', '18', '19'],
      dtype='object')

In [9]:
df_mdb.tail()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
4084,11004958,Nölke,Matthias,08.01.1980,Kassel,NaN,NaN,männlich,ledig,konfessionslos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4085,11004959,Martin,Dorothee,21.01.1978,NaN,NaN,NaN,weiblich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4086,11004960,Wetzel,Wolfgang,11.05.1968,Schlema,NaN,NaN,männlich,"verheiratet, 2 Kinder",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4087,11004961,Natterer,Christian,19.02.1981,Tettnang / Deutschland,NaN,NaN,männlich,"verheiratet, 1 Kind",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4088,11004962,Dahmen,Janosch,06.09.1981,NaN,NaN,NaN,männlich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [58]:
df_mdb_wp[['ID', 'PARTEI_KURZ']].groupby('PARTEI_KURZ').count().sort_values(by='ID', ascending=False).head(10)

,ID
PARTEI_KURZ,
SPD,4250
CDU,4150
FDP,1036
CSU,993
DIE GRÜNEN,536
DIE LINKE.,297
AfD,90
PDS,77
Plos,41


In [59]:
# wie - nur 2 Geschlechter? for the records: heute ist der 02.05.2021
df_mdb_wp[['ID', 'GESCHLECHT']].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).head()

,ID
GESCHLECHT,
männlich,9382
weiblich,2245


In [60]:
# auch hier: such diversity. wow
df_mdb_wp[['ID', 'RELIGION']].groupby('RELIGION').count().sort_values(by='ID', ascending=False).head(n=16)

,ID
RELIGION,
evangelisch,3406
ohne Angaben,3275
katholisch,2985
römisch-katholisch,831
evangelisch-lutherisch,366
konfessionslos,234
evangelisch-reformiert,34
Atheistin,12
Islam,9


In [62]:
# jetzt wirds richtig vielseitig. so viele Busfahrer und Krankenschwestern!
df_mdb_wp[['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head(16).head(16)

,ID
BERUF,
Rechtsanwalt,567
Geschäftsführer,223
Landwirt,212
Angestellter,190
Rechtsanwalt und Notar,126
Journalist,117
Dipl.-Ingenieur,109
Kaufmann,104
Jurist,102


In [63]:
# gehen sie weiter. hier gibt es kein Klischee zu erkennen!
df_mdb_wp[['ID', 'FAMILIENSTAND']].groupby('FAMILIENSTAND').count().sort_values(by='ID', ascending=False).head(16)

,ID
FAMILIENSTAND,
"verheiratet, 2 Kinder",2726
"verheiratet, 3 Kinder",1816
verheiratet,1720
"verheiratet, 1 Kind",1365
keine Angaben,1043
"verheiratet, 4 Kinder",891
ledig,424
"verheiratet, 5 Kinder",329
"verheiratet, 6 Kinder",178


In [143]:
# deleteme
start_date=1
end_date=17
#wps = range(start_date, end_date)
#selected_df = pd.concat([df_mdb[df_mdb[str(i)] == 1] for i in range(start_date,end_date+1)]).drop_duplicates()# deleteme
selected_df = df_mdb_wp[(df_mdb_wp['WP']>= start_date) & (df_mdb_wp['WP']<= end_date)]
selected_df.head()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,ANZ_WAHLPERIODEN,WP
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
1,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
2,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
3,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11
4,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,7,11


In [137]:
# deleteme
grouped = selected_df[['ID', 'WP', 'GESCHLECHT']].groupby(['GESCHLECHT', 'WP']).count()
grouped

ID
GESCHLECHT WP     
männlich   1   190
           2   255
           3   280
           4   443
           5   513
           6   514
           7   395
           8   520
           9   305
           10  457
           11  641
           12  681
           13  522
           14  622
           15  350
           16  494
           17  520
weiblich   1    14
           2    21
           3    31
           4    44
           5    64
           6    65
           7    21
           8    32
           9    19
           10   35
           11  111
           12   81
           13   90
           14  200
           15  160
           16  183
           17  221

In [122]:
grouped.reset_index(inplace=True)
grouped

,GESCHLECHT,WP,ID
0,männlich,2,255
1,männlich,3,280
2,männlich,4,443
3,männlich,5,513
4,männlich,6,514
5,männlich,7,395
6,weiblich,2,21
7,weiblich,3,31
8,weiblich,4,44
9,weiblich,5,64


In [130]:
# deleteme
wps = list(range(start_date, end_date +1))
geschlechter = list(set(grouped.GESCHLECHT))
traces_values = []

for geschlecht in geschlechter:
    trace = grouped[grouped['GESCHLECHT'] == geschlecht].sort_values(by='WP').ID.values
    traces_values.append(trace)
traces_values

[array([255, 280, 443, 513, 514, 395]), array([21, 31, 44, 64, 65, 21])]

In [135]:
print('\n' , 3)


 3


In [132]:
traces = [go.Bar(x=wps, y=trace, xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name=f'{gender}') for trace, gender in zip(traces_values, geschlechter)]
traces

[Bar({
     'marker': {'color': '#0099ff'},
     'name': 'männlich',
     'x': [2, 3, 4, 5, 6, 7],
     'xaxis': 'x2',
     'y': array([255, 280, 443, 513, 514, 395]),
     'yaxis': 'y2'
 }),
 Bar({
     'marker': {'color': '#0099ff'},
     'name': 'weiblich',
     'x': [2, 3, 4, 5, 6, 7],
     'xaxis': 'x2',
     'y': array([21, 31, 44, 64, 65, 21]),
     'yaxis': 'y2'
 })]

In [133]:
myfigure = {'data': traces,
        'layout': go.Layout(title = 'My plot',
        xaxis = {'title' : 'WP', 'type': 'log'},
        yaxis = {'title' : 'gender per WP'})}

In [119]:
fig = dcc.Figure(myfigure)

NameError: name 'dcc' is not defined

In [113]:
#import plotly.figure_factory as ff
# does not work / does not show
fig = go.Figure()
fig.add_traces(traces)
fig.layout.update({'title': 'gender'})
fig.show()

In [18]:
# deleteme
page_current = 0
page_size = 4
COLUMNS_FOR_DISPLAY = ['NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'PARTEI_KURZ', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION', 'BERUF', 'VITA_KURZ']
    
selected_df.iloc[
        page_current*page_size:(page_current+ 1)*page_size
    ][COLUMNS_FOR_DISPLAY].to_dict('records')

[{'NACHNAME': 'Ackermann',
  'VORNAME': 'Annemarie',
  'GEBURTSDATUM': '26.05.1913',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'weiblich',
  'FAMILIENSTAND': 'verheiratet, 5 Kinder',
  'RELIGION': 'katholisch',
  'BERUF': 'Hilfsreferentin',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Adenauer',
  'VORNAME': 'Konrad',
  'GEBURTSDATUM': '05.01.1876',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'männlich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'katholisch',
  'BERUF': 'Bundeskanzler a. D.',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Albers',
  'VORNAME': 'Johannes',
  'GEBURTSDATUM': '08.03.1890',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'männlich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'katholisch',
  'BERUF': 'Geschäftsführer',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Albertz',
  'VORNAME': 'Luise',
  'GEBURTSDATUM': '22.06.1901',
  'PARTEI_KURZ': 'SPD',
  'GESCHLECHT': 'weiblich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'ohne Angaben',
  'BERUF': 'Buchhalterin',
  'VITA_KURZ': n

In [29]:
# deleteme
#! pip3 install plotly
from plotly import graph_objs as go
fig = go.Figure()
hist = go.Histogram(histfunc="count",  x=selected_df['GESCHLECHT'])
hist

Histogram({
    'histfunc': 'count',
    'x': array(['weiblich', 'männlich', 'männlich', ..., 'männlich', 'weiblich',
                'männlich'], dtype=object)
})

In [32]:
# deleteme
selected_df[['ID', 'GESCHLECHT', ]].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).head(16)

,ID
GESCHLECHT,
männlich,625
weiblich,66


In [39]:
# deleteme
columns_to_keep = selected_df.columns[:15]
list_of_entries = []
for row in selected_df:
    
columns_to_keep

Index(['ID', 'NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'GEBURTSORT',
       'GEBURTSLAND', 'STERBEDATUM', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION',
       'BERUF', 'PARTEI_KURZ', 'VITA_KURZ', 'VEROEFFENTLICHUNGSPFLICHTIGES',
       'ANZ_WAHLPERIODEN'],
      dtype='object')